<a href="https://colab.research.google.com/github/rohithmada00/BYG/blob/main/ticc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Install necessary dependencies
# !pip3 install snap-stanford

# # Clone the SNAP repository
# !git clone https://github.com/snap-stanford/snap-python.git
# !git clone https://github.com/snap-stanford/snap.git

# # Move into the SNAP directory and build from source
# %cd snap-python
# !make

In [ ]:
# import necessary libraries

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.cluster  import KMeans
import matplotlib.pyplot as plt
import tqdm
import collections
import random

In [ ]:
# @title Generate synthetic data
# Parameters to play with
window_size = 5
number_of_sensors = 5
sparsity_inv_matrix = 0.2
rand_seed = 10
number_of_clusters = 3
cluster_ids = [0, 1, 0]
break_points = np.array([1, 2, 3]) * 200
save_inverse_covariances = True
out_file_name = "Synthetic_Data_Matrix_rand_seed=[0,1]_generated2.csv"

block_matrices = {}  # Stores all the block matrices
num_blocks = window_size
size_blocks = number_of_sensors
seg_ids = cluster_ids

def generate_inverse(rand_seed):
    np.random.seed(rand_seed)

    def gen_inv_cov(size, low=0.3, upper=0.6, portion=0.2, symmetric=True):
        S = np.zeros((size, size))
        mask = random(size, size, density=portion, data_rvs=uniform(loc=low, scale=upper - low).rvs, random_state=rand_seed).toarray()
        mask = np.triu(mask, 1)  # Get the upper triangular part
        S += mask
        S += mask.T  # Make it symmetric
        return np.matrix(S)

    def gen_rand_inv(size, low=0.3, upper=0.6, portion=0.2):
        S = np.zeros((size, size))
        for i in range(size):
            for j in range(size):
                if np.random.rand() < portion:
                    value = (np.random.randint(2) - 0.5) * 2 * (low + (upper - low) * np.random.rand(1)[0])
                    S[i, j] = value
        return np.matrix(S)

    # Generate all the blocks
    for block in range(num_blocks):
        if block == 0:
            block_matrices[block] = gen_inv_cov(size=size_blocks, portion=sparsity_inv_matrix, symmetric=(block == 0))
        else:
            block_matrices[block] = gen_rand_inv(size=size_blocks, portion=sparsity_inv_matrix)

    # Initialize the inverse matrix
    inv_matrix = np.zeros([num_blocks * size_blocks, num_blocks * size_blocks])

    # Go through all the blocks
    for block_i in range(num_blocks):
        for block_j in range(num_blocks):
            block_num = np.abs(block_i - block_j)
            if block_i > block_j:
                inv_matrix[block_i * size_blocks:(block_i + 1) * size_blocks, block_j * size_blocks:(block_j + 1) * size_blocks] = block_matrices[block_num]
            else:
                inv_matrix[block_i * size_blocks:(block_i + 1) * size_blocks, block_j * size_blocks:(block_j + 1) * size_blocks] = block_matrices[block_num].T

    # Print out all the eigenvalues
    eigs, _ = np.linalg.eig(inv_matrix)
    lambda_min = min(eigs)

    # Make the matrix positive definite
    inv_matrix = inv_matrix + (0.1 + abs(lambda_min)) * np.identity(size_blocks * num_blocks)

    eigs, _ = np.linalg.eig(inv_matrix)
    lambda_min = min(eigs)
    print("Modified Eigenvalues are:", np.sort(eigs))

    return inv_matrix

# Generate Points
num_clusters = number_of_clusters
cluster_mean = np.zeros([size_blocks, 1])
cluster_mean_stacked = np.zeros([size_blocks * num_blocks, 1])

# Generate two inverse matrices
cluster_inverses = {}
cluster_covariances = {}
for cluster in range(num_clusters):
    cluster_inverses[cluster] = generate_inverse(rand_seed=cluster)
    cluster_covariances[cluster] = np.linalg.inv(cluster_inverses[cluster])
    if save_inverse_covariances:
        np.savetxt(f"Inverse_Covariance_cluster={cluster}.csv", cluster_inverses[cluster], delimiter=",", fmt='%1.6f')
        np.savetxt(f"Covariance_cluster={cluster}.csv", cluster_covariances[cluster], delimiter=",", fmt='%1.6f')

print("Done till this!!")

# Data matrix
Data = np.zeros([break_points[-1], size_blocks])
for counter in range(len(break_points)):
    break_pt = break_points[counter]
    cluster = seg_ids[counter]
    old_break_pt = 0 if counter == 0 else break_points[counter - 1]
    for num in range(old_break_pt, break_pt):
        cov_matrix = cluster_covariances[cluster]
        new_mean = cluster_mean_stacked[size_blocks * (num_blocks - 1):size_blocks * num_blocks]
        new_row = np.random.multivariate_normal(new_mean.reshape(size_blocks), cov_matrix[:size_blocks, :size_blocks])
        Data[num, :] = new_row

print("Done with generating the data!!!")
print("Length of generated Data is:", Data.shape[0])

# Save the generated matrix
np.savetxt(out_file_name, Data, delimiter=",", fmt='%1.4f')

In [ ]:
# set parameters

window_size = 10 # number of seconds in the timeframe to consider as a block for a cluster
max_iters = 100 # number of iterations before convergence
beta = 5 # smoothness penalty for switching between clusters ; higher value penalizes rapid changes between clusters
lambda_param = 11e-2 # regularizer for sparsity in inverse covariance matrix ; higher value encourages sparser covariance matrix
k = 2 # number of clusters
threshold = 2e-5 # precision threshold for convergence ; lower value encourages greater precision ; may slow down convergence

Data Analysis and Train - Test split


In [ ]:
# fetch the dataset
df = pd.read_csv("/content/ticc_data.txt", header=None)

In [ ]:
df.shape

(19607, 10)

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9
0,1.689432,4.242490,-0.255901,-0.045349,-0.139132,-0.114547,-0.360088,0.285042,-0.048519,0.041641
1,1.696751,4.250964,-0.195189,0.156340,0.022974,-0.089657,-0.324283,-0.002790,0.115144,0.008604
2,1.685673,4.260681,-0.179823,0.145138,-0.009416,-0.075564,-0.397804,0.057725,0.023173,0.032430
3,1.711946,4.287914,-0.247423,0.027780,-0.128066,-0.093688,-0.247294,0.165290,0.039309,0.179431
4,1.704324,4.255675,-0.108945,0.084931,0.184524,-0.015141,-0.265052,0.311020,0.050675,0.145509


In [ ]:
df.describe()

,0,1,2,3,4,5,6,7,8,9
count,1.960700e+04,1.960700e+04,1.960700e+04,1.960700e+04,1.960700e+04,1.960700e+04,1.960700e+04,1.960700e+04,1.960700e+04,1.960700e+04
mean,-2.783173e-16,-2.319311e-17,1.884440e-16,-5.798278e-18,2.826661e-16,-1.623518e-16,-2.783173e-16,-1.877193e-16,2.464268e-16,-2.551242e-16
std,3.724569e+00,2.074234e+00,9.084122e-01,7.861404e-01,7.408796e-01,6.993198e-01,5.764718e-01,5.560582e-01,5.457726e-01,5.223418e-01
min,-4.341914e+00,-3.243643e+00,-3.017925e+00,-2.435382e+00,-2.919251e+00,-2.088742e+00,-1.692283e+00,-2.278562e+00,-2.001753e+00,-1.577318e+00
25%,-3.754294e+00,-1.139824e+00,-3.290734e-01,-3.882132e-01,-4.186700e-01,-3.973303e-01,-4.092253e-01,-2.146821e-01,-3.030167e-01,-3.550449e-01
50%,8.502990e-01,-6.914171e-01,-1.566461e-01,5.138096e-02,-3.739585e-02,-1.268217e-01,-1.475935e-01,2.103342e-02,1.562663e-02,-8.059647e-03
75%,4.384427e+00,3.156790e-01,1.737884e-01,3.008897e-01,3.373168e-01,3.183570e-01,2.494907e-01,1.925995e-01,2.445546e-01,2.415141e-01
max,5.484738e+00,4.338761e+00,5.886119e+00,3.260147e+00,3.260171e+00,3.151718e+00,2.980588e+00,2.656090e+00,4.345893e+00,2.837008e+00


In [ ]:
df.isnull().sum().sum()

0

In [ ]:
# Split data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

Windowing the time series data

In [ ]:
def window_data(data :np.ndarray, window_size: int = 10):
    n_samples, n_features = data.shape
    windowed_data = np.zeros((n_samples - window_size + 1, window_size * n_features))
    for i in range(n_samples - window_size + 1):
        windowed_data[i] = data[i:i + window_size].flatten()
    return windowed_data

In [ ]:
# convert into np array to make use of array slicing
train_data = train_df.values
test_data = test_df.values

windowed_train_data = window_data(train_data, window_size)
windowed_test_data = window_data(test_data, window_size)

In [ ]:
print(f"Shape of windowed train data: {windowed_train_data.shape}")
print(f"Shape of windowed test data: {windowed_test_data.shape}")

Shape of windowed train data: (15676, 100)
Shape of windowed test data: (3913, 100)


Baseline: K-Means Clustering

In [ ]:
kmeans = KMeans(n_clusters=k, random_state=42).fit(windowed_train_data)
cluster_labels_train = kmeans.labels_
cluster_labels_test = kmeans.predict(windowed_test_data)

# TODO: compute a confusion matrix for this

In [ ]:
print(f"Train cluster distribution: {np.bincount(cluster_labels_train)}")
print(f"Test cluster distribution: {np.bincount(cluster_labels_test)}")

Train cluster distribution: [7485 8191]
Test cluster distribution: [1874 2039]


In [ ]:
# import matplotlib.pyplot as plt

# # Plot training cluster labels
# plt.figure(figsize=(12, 6))
# plt.plot(cluster_labels_train, label='Train Clusters')
# plt.title("Train Cluster Assignments")
# plt.xlabel("Window Index")
# plt.ylabel("Cluster Label")
# plt.legend()
# plt.show()

# # Plot testing cluster labels
# plt.figure(figsize=(12, 6))
# plt.plot(cluster_labels_test, label='Test Clusters', color='orange')
# plt.title("Test Cluster Assignments")
# plt.xlabel("Window Index")
# plt.ylabel("Cluster Label")
# plt.legend()
# plt.show()

TICC

In [ ]:
n_samples, n_features = train_data.shape

# Initialization
cluster_assignment = np.random.randint(k, size=len(windowed_train_data))
cluster_covariance = [np.identity(window_size * n_features) for _ in range(k)]
cluster_mean = [np.mean(windowed_train_data[cluster_assignment == i], axis=0) for i in range(k)]

In [ ]:
# our aim is to optimize the cluster covariance

import numpy as np
from sklearn.covariance import graphical_lasso

def m_step(windowed_data, cluster_assignments, num_clusters, lambda_param):
    """
    Perform the M-step of the TICC algorithm by updating the Toeplitz inverse covariance
    matrices for each cluster.

    Parameters:
    - windowed_data: np.ndarray of shape (num_windows, window_size * num_features),
      flattened windows of the time series data.
    - cluster_assignments: np.ndarray of shape (num_windows,), cluster assignments for each window.
    - num_clusters: int, number of clusters (K).
    - lambda_param: float, regularization parameter for sparsity.

    Returns:
    - inverse_covariances: list of np.ndarray, the updated inverse covariance matrices for each cluster.
    - means: list of np.ndarray, the updated means for each cluster.
    """
    num_features = windowed_data.shape[1]
    inverse_covariances = []
    means = []

    for cluster in range(num_clusters):
        # Get data points assigned to this cluster
        cluster_data = windowed_data[cluster_assignments == cluster]

        if len(cluster_data) == 0:
            # Handle empty cluster case (rare but possible)
            inverse_covariances.append(np.eye(num_features))
            means.append(np.zeros(num_features))
            continue

        # Compute the empirical covariance matrix for the cluster
        empirical_cov = np.cov(cluster_data, rowvar=False)

        # Solve the graphical lasso for sparse inverse covariance
        cov, inv_cov = graphical_lasso(empirical_cov, alpha=lambda_param)

        # Store the results
        inverse_covariances.append(inv_cov)
        means.append(np.mean(cluster_data, axis=0))

    return inverse_covariances, means


In [ ]:
def assign_cluster(lle, beta_param):
    """
    Assign clusters with temporal consistency using dynamic programming.

    Parameters:
    - lle: 2D numpy array of log-likelihood values (T x K), where lle[i][j] is the negative
      log-likelihood of assigning point i to cluster j.
    - beta_param: Smoothness penalty for switching clusters.

    Returns:
    - Optimal cluster assignment path.
    """
    t, k = lle.shape
    # Initialize costs and paths
    prev_cost = [0] * k
    prev_path = [[j] for j in range(k)]

    for i in range(t):
        curr_cost = [0] * k
        curr_path = [[] for _ in range(k)]

        for j in range(k):
            # Calculate cost for staying in the same cluster
            stay_cost = prev_cost[j] - lle[i][j]

            # Calculate cost for switching to a different cluster
            switch_cost = [prev_cost[m] + beta_param - lle[i][j] for m in range(k)]
            min_switch_cost = min(switch_cost)

            # Choose the better option: stay or switch
            if stay_cost <= min_switch_cost:
                curr_cost[j] = stay_cost
                curr_path[j] = prev_path[j] + [j]
            else:
                min_switch_index = np.argmin(switch_cost)
                curr_cost[j] = min_switch_cost
                curr_path[j] = prev_path[min_switch_index] + [j]

        # Update for next iteration
        prev_cost = curr_cost
        prev_path = curr_path

    # Return the path with the minimum cost
    return prev_path[np.argmin(prev_cost)]

In [ ]:
import numpy as np
import tqdm

# Initialize variables
cluster_assignment = np.random.randint(0, k, size=len(windowed_train_data))  # Random initial cluster assignments
cluster_covariance = [np.eye(windowed_train_data.shape[1]) for _ in range(k)]  # Initialize covariance matrices
cluster_mean = [np.zeros(windowed_train_data.shape[1]) for _ in range(k)]  # Initialize means

# TICC iterations
for iteration in tqdm.tqdm(range(5)):
    # M-step: Optimize covariance and mean for each cluster
    cluster_covariance, cluster_mean = m_step(windowed_train_data, cluster_assignment, k, lambda_param)

    # E-step: Calculate log-likelihood for each point-cluster pair
    lle = np.zeros((len(windowed_train_data), k))
    for i in range(len(windowed_train_data)):
        for j in range(k):
            cov_inv = np.linalg.inv(cluster_covariance[j])  # Inverse covariance
            mean_diff = windowed_train_data[i] - cluster_mean[j]  # (X - mean)
            lle[i][j] = (
                -0.5 * mean_diff.T @ cov_inv @ mean_diff
                + 0.5 * np.log(np.linalg.det(cov_inv))
                - (len(windowed_train_data) * np.log(2 * np.pi))
            )

    # Assign clusters with temporal consistency
    new_cluster_assignment = assign_cluster(lle, beta)

    # Check for convergence
    if np.array_equal(cluster_assignment, new_cluster_assignment):
        print(f"Converged after {iteration + 1} iterations.")
        break

    # Update cluster assignments
    cluster_assignment = new_cluster_assignment

print("Final Cluster Assignment:", np.unique(cluster_assignment))


 40%|████      | 2/5 [01:30<02:16, 45.34s/it]

Converged after 3 iterations.
Final Cluster Assignment: [0]
